In [130]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib
from scipy.optimize import fsolve

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [132]:
specs = loaders.load_density()
specs = geography.get_coordinate_functions(specs)
airports = loaders.load_airports(specs)
debug = True
if debug:
    airports = airports.loc[vis.airport_list]
vis.plot_airports(airports, specs['data'])
vis.plot_density(specs['data'])

travel = loaders.load_travel(airports)
valid = airports.NodeName.values
travel['annual'] = travel['annual'].query('Origin in @valid and Dest in @valid')

# kappas = [1, 1, 2, 5]
# infected = [1, 2, 1, 1]
for kappa, n in zip(kappas, infected):
    airports = loaders.calculate_outbreaks(airports, kappa=kappa, n=n)
    travel['annual'] = loaders.augment_travel(travel['annual'], airports)
    # travel = {time: augment_travel(df, airports) for time, df in travel.items()}
    vis.plot_risks(travel['annual'], n=n, kappa=kappa)

vis.plot_geodesics(travel['annual'])


Plotting airports
Plotting density
Plotting risks
Plotting risks


/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Plotting risks
Plotting risks
Plotting geodesics


In [129]:
travel['annual']

,Origin,Dest,Prediction,lower,upper,origin_p_outbreak,dest_p_outbreak,outgoing_total,P_ij,risk_ij,risk_i,origin_lon,origin_lat,dest_lon,dest_lat,red,blue,green
0,CDG,DEL,9634.333333,9126.250000,10301.416667,7.229794e-01,8.250972e-01,260287.233333,0.037014,0.006474,0.556644,2.57,49.00,77.10,28.56,0.556644,0.443356,0
1,CDG,EWR,15015.000000,14327.916667,15736.583333,7.229794e-01,7.613931e-01,260287.233333,0.057686,0.013764,0.556644,2.57,49.00,-74.18,40.69,0.556644,0.443356,0
2,CDG,FRA,50677.916667,48206.833333,53283.416667,7.229794e-01,-2.220446e-16,260287.233333,0.194700,0.194700,0.556644,2.57,49.00,8.57,50.05,0.556644,0.443356,0
3,CDG,JFK,46421.750000,44784.750000,48122.333333,7.229794e-01,7.613931e-01,260287.233333,0.178348,0.042555,0.556644,2.57,49.00,-73.79,40.64,0.556644,0.443356,0
4,CDG,LHR,57249.083333,54726.500000,59895.583333,7.229794e-01,6.696552e-01,260287.233333,0.219946,0.072658,0.556644,2.57,49.00,-0.45,51.47,0.556644,0.443356,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,TXL,JFK,4245.833333,4103.666667,4393.416667,-6.661338e-16,7.613931e-01,150702.181818,0.028174,0.006722,0.703069,13.29,52.55,-73.79,40.64,0.703069,0.296931,0
130,TXL,LHR,24806.750000,23857.916667,25798.000000,-6.661338e-16,6.696552e-01,150702.181818,0.164608,0.054377,0.703069,13.29,52.55,-0.45,51.47,0.703069,0.296931,0
131,TXL,MIA,2240.500000,1948.000000,2579.500000,-6.661338e-16,3.829999e-01,150702.181818,0.014867,0.009173,0.703069,13.29,52.55,-80.28,25.80,0.703069,0.296931,0
132,TXL,ORY,7691.090909,7425.090909,7967.909091,-6.661338e-16,7.229794e-01,150702.181818,0.051035,0.014138,0.703069,13.29,52.55,2.36,48.73,0.703069,0.296931,0


In [111]:
x1 = dd.Prediction.values
x2 = 1000 * np.ones(len(dd))

np.maximum(x1, x2)

array([ 2691.66666667,  9737.83333333,  6361.08333333,  8640.58333333,
        1000.        ,  1090.16666667,  1000.        , 10102.25      ,
        1000.        ,  3600.6       ,  1000.        ,  1000.        ,
        1434.25      ,  4450.        ,  2713.75      ,  2615.75      ,
        3506.66666667,  1349.16666667,  6573.16666667,  1000.        ,
        9645.5       ,  2575.5       ,  2647.66666667, 10430.91666667,
       16670.        ,  2295.58333333,  1000.        ,  8967.33333333,
        8713.5       ,  1000.        , 10010.        , 23607.08333333,
        1000.        , 11986.5       ,  1798.4       ,  1000.        ,
        8785.75      ,  1000.        ,  7815.16666667, 12835.08333333,
        1000.        , 10410.91666667,  1200.        , 16747.91666667,
        1482.5       ,  3105.58333333,  1374.25      ,  7663.        ,
        1292.66666667,  1000.        , 17161.16666667,  1241.25      ,
        1000.        ,  1000.        ,  1000.        ,  4630.58333333,
      

In [110]:
x2.shape

(100,)